***Color manual***

<div class="alert alert-block alert-success">
<b>Green - Libraries</b>
</div>

<div class="alert alert-block alert-info">
<b>Blue - Information</b>
</div>

<div class="alert alert-block alert-danger">
<b>Red - Problems</b>
</div>

***

<div class="alert alert-block alert-success">
<b>Import libraries</b>
</div>

In [9]:
import numpy as np
import chart_studio
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data

# If sklearn is needed 
from sklearn.linear_model import LinearRegression

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

# To remove date from index 
# reset_index(drop = False, inplace = True)

# To erase NaN values
# dropna()

<div class="alert alert-block alert-info">
<b>Load the historical data, transform in montly data and get percentage change</b>
</div>

In [2]:
# If working for any with a list like ['Stock name'....] know that the work flow is different
# R = pdr.DataReader(['FLRY3.SA'], 'yahoo', start = '2010/01/01', end = '2019/12/31')

R = data.DataReader('FLRY3.SA', 'yahoo', start = '2017/01/01', end = '2019/12/31')
R = R.resample('M').last().pct_change()
R.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-28,0.142783,0.163651,0.178456,0.140600,0.115421,0.140600
2017-03-31,-0.028956,-0.025992,-0.033677,-0.023546,-0.158339,-0.001877
2017-04-30,0.249354,0.211103,0.205074,0.241608,1.062273,0.241608
2017-05-31,0.084759,0.103679,0.130828,0.080922,-0.596656,0.080921


<div class="alert alert-block alert-info">
<b>Same as above but with expected return of the market with GSPC = S&P 500 </b>
</div>

In [3]:
Rm = data.DataReader('^GSPC', 'yahoo', start = '2017/01/01', end = '2019/12/31')
Rm = Rm.resample('M').last().pct_change()
Rm.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-28,0.038919,0.040468,0.040483,0.037198,0.030016,0.037198
2017-03-31,0.001081,0.001543,-0.000533,-0.000389,-0.203326,-0.000389
2017-04-30,0.009842,0.008364,0.012204,0.009091,0.108571,0.009091
2017-05-31,0.009320,0.008911,0.009170,0.011576,0.214573,0.011576


<div class="alert alert-block alert-info">
<b>Same as above but with risk-free investment with DGS1MO = One Month Treasury Constant Maturity Rate</b>
</div>

In [4]:
Rf = data.DataReader('DGS1MO', 'fred', start = '2017/02/01', end = '2019/12/31')
Rf = Rf.resample('M').last()
Rf = Rf/100
Rf.head()

,DGS1MO
DATE,
2017-02-28,0.0040
2017-03-31,0.0074
2017-04-30,0.0068
2017-05-31,0.0086
2017-06-30,0.0084


<div class="alert alert-block alert-info">
<b>Table for Capital Asset Pricing Model</b>
</div>

In [5]:
# When working with a list it's good to use concat method 
# CAPM = pd.concat([R['Adj Close'], Rm['Adj Close'], Rf['DGS1MO']], axis = 1)
# CAPM.rename(columns = {'FLRY3.SA': 'R', '^GSPC': 'Rm', 'DGS1MO': 'Rf'}, inplace = True)

CAPM = pd.DataFrame({'R': R['Adj Close'], 'Rm': Rm['Adj Close'], 'Rf': Rf['DGS1MO']}).dropna()
CAPM['R - Rf'] = CAPM['R'] - CAPM['Rf']
CAPM['Rm - Rf'] = CAPM['Rm'] - CAPM['Rf']
CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf
Date,,,,,
2017-02-28,0.140600,0.037198,0.0040,0.136600,0.033198
2017-03-31,-0.001877,-0.000389,0.0074,-0.009277,-0.007789
2017-04-30,0.241608,0.009091,0.0068,0.234808,0.002291
2017-05-31,0.080921,0.011576,0.0086,0.072321,0.002976
2017-06-30,-0.055839,0.004814,0.0084,-0.064239,-0.003586


<div class="alert alert-block alert-info">
<b>Linear regression</b>
</div>

In [6]:
x = CAPM['Rm - Rf']
y = CAPM['R - Rf']

x = sm.add_constant(x)
model = sm.OLS(y, x, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 R - Rf   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     5.907
Date:                Mon, 04 May 2020   Prob (F-statistic):             0.0207
Time:                        22:21:39   Log-Likelihood:                 37.876
No. Observations:                  35   AIC:                            -71.75
Df Residuals:                      33   BIC:                            -68.64
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0099      0.014      0.685      0.498      -0.020       0.039
Rm - Rf        0.9596      0.395      2.430      0.021       0.156       1.763
==============================================================================
Omnibus:                        0.640   Durbin-Watson:                   2.071
Prob(Omnibus):                  0.726   Jarque-Bera (JB):                0.088
Skew:                           0.043   Prob(JB):                        0.957
Kurtosis:                       3.230   Cond. No.                         27.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<div class="alert alert-block alert-info">
<b>Add deviation from the mean, predicted values, simple residuals, squared residuals and standard error of estimate to table</b>
</div>

In [7]:
# Deviation from the mean
mean = CAPM['R - Rf'].mean()
CAPM['Deviation from mean'] = CAPM['R - Rf'] - mean

# Predicted values
y_pred = results.predict()
CAPM['Predicted'] = y_pred

# Simple residuals
residual = results.resid
CAPM['Residual'] = residual

# Squared residuals
squared_residual = results.ssr
CAPM['Squared residual'] = squared_residual

# Standard error of estimate
x_pred = np.linspace(CAPM['Rm - Rf'].min(), CAPM['Rm - Rf'].max(), 35)
x_pred2 = sm.add_constant(x_pred)
SEE = results.predict(x_pred2)
CAPM['SEE'] = SEE

CAPM.head()

,R,Rm,Rf,R - Rf,Rm - Rf,Deviation from mean,Predicted,Residual,Squared residual,SEE
Date,,,,,,,,,,
2017-02-28,0.140600,0.037198,0.0040,0.136600,0.033198,0.132247,0.041764,0.094837,0.235305,-0.101578
2017-03-31,-0.001877,-0.000389,0.0074,-0.009277,-0.007789,-0.013631,0.002432,-0.011709,0.235305,-0.096761
2017-04-30,0.241608,0.009091,0.0068,0.234808,0.002291,0.230454,0.012105,0.222703,0.235305,-0.091944
2017-05-31,0.080921,0.011576,0.0086,0.072321,0.002976,0.067968,0.012762,0.059559,0.235305,-0.087128
2017-06-30,-0.055839,0.004814,0.0084,-0.064239,-0.003586,-0.068592,0.006465,-0.070704,0.235305,-0.082311


<div class="alert alert-block alert-info">
<b>CAPM plot</b>
</div>

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = y_pred, 
    name = 'Linear regression', 
    mode = 'lines',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(145, 255, 255)')))

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = CAPM['R - Rf'], 
    name = 'Actual values', 
    mode = 'markers',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 255, 145)')))

fig.add_trace(go.Scatter(x = CAPM['Rm - Rf'], y = SEE, 
    name = 'Standard error of estimate', 
    mode = 'markers',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 145, 255)')))

fig.update_layout(title_text = 'CAPM - FLRY3 and S&P 500 (2017 - 2019)', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(x = 0, y = 1.5),
    xaxis_title = 'Beta',
    yaxis_title = 'Monthly expected return',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel= dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, filename = 'Capital Asset Pricing Model using FLRY3 and S&P 500', auto_open = False)

$CAPM$
$$Y_i = b_0 + b_1 X_i + \epsilon_i$$ <br/>
$$(R\ -\ Rf)\ =\ b_{0}\ +\ b_{1}(Rm\ -\ Rf)\ +\ \epsilon$$

$Standard\ error\ of\ estimate$
$$SEE = \Bigg( \frac{\sum_{i=1}^n (Y_i - \hat{b}_0-\hat{b}_1X_i)^2}{n-2} \Bigg)^\frac{1}{2} = \Bigg( \frac{\sum_{i=1}^n (\hat\epsilon_i)^2}{n-2} \Bigg)^\frac{1}{2}$$